# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico III : Introducción al aprendizaje automático

**Antes de empezar:**
- [Instalar spaCy y el modelo de lenguaje con el que van a trabajar](https://spacy.io/models#quickstart)

**Consideraciones:**
- Se evalúa el estilo y prolijidad del código.
- Se permite hacer trabajo extra, siempre y cuando las actividades básicas estén resueltas.

**Recomendación:**
- Hay muchos ejemplos de código en internet, no se sientan obligados a implementar todo desde cero.

### Librerías

In [ ]:
!pip3 install spotipy
!pip3 install pandas
!pip3 install spacy
!pip3 install pymusixmatch
!pip3 install nltk
!pip3 install sklearn

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

### Dependencias y acceso a APIs

In [5]:
import pandas as pd
import numpy as np
import spotipy
import spacy
from spotipy.oauth2 import SpotifyClientCredentials
from musixmatch import Musixmatch
import seaborn as sns
import tqdm
import plotly.express as px
import plotly.graph_objects as go

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# es = es_core_web_sm
nlp = spacy.load("es") # completar con el modelo que van a utilizar

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
musixmatch = Musixmatch('1aa5272f4402bf2f082ad2f3958c2c62') # se puede reemplazar por otra API si da mejores resultados

In [4]:
sns.set_context(context='paper')

### 1) Recopilar los datos obtenidos en los prácticos anteriores

Para esta parte consideraremos [la playlist colaborativa de la mentoría](https://open.spotify.com/playlist/2IuD0qZb14cji5y52crdsO?si=nfHRPDquQRyotEcXc4tG7Q), de esta obtendremos:
- Las features del audio de las canciones
- Las features textuales de sus letras

Además es necesario aplicar el mismo preprocesamiento que aplicamos en los prácticos anteriores para ambos tipos de features (el preprocesamiento del p1 a las features de audio y el de p2 al de features textuales) y obtener el género de cada canción, que en caso de ser más de uno para una canción el equipo deberá discutir una estrategia para estos casos y comentarla en el informe.

Luego, se separará al dataset resultante en **X** e **y**, donde:
- X es el conjunto de features
- y es la etiqueta, en este caso el género de la canción, que deberá ser codificado en valores del tipo **int**

Por último, se dividirá a estos dos conjuntos en los splits **train** y **test**

**Recomendaciones:**
- Obtener las features por separado y hacer un join de los datasets.
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)
- Si usan features categóricas, ENCODEARLAS!

# Dataset

## Music Features

In [6]:
#Aux funcs
def genres_by_artist_id(id): #id: str
    artist = sp.artist(id)
    genres = artist['genres']
    return genres    #genres: List[str]

def songs_from_album_id(album_id):
    songs = []
    album = sp.album(album_id)
    artist = album['artists'][0]['name']
    for item in album['tracks']['items']:
        track = {}
        track["song_name"] = item['name']
        track["song_id"] = item['id']
        track["album_name"] = album['name']
        track["album_id"] = album["id"]
        audio_features = sp.audio_features(track["song_id"])
        track["audio_features"] = audio_features[0]
        track["artist"] = artist
        songs.append(track)
    return songs    #songs:List[dict]

def add_track(track_id, songs): #track_id:str, songs:List[dict]
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)
    row = {}
    row["song_name"] = track['name']
    row["song_id"] = track['id']
    row["album_name"] = track['album']['name']
    row["album_id"] = track['album']['id']
    row["audio_features"] = audio_features[0]
    songs.append(row)
    return songs


In [103]:
#dir(sp)
#sp.audio_analysis()
PLAYLIST_ID = "2IuD0qZb14cji5y52crdsO"
songs = []
#playlist = sp.playlist("2IuD0qZb14cji5y52crdsO")
def add_songs_of_playlist(playlist_id,songs_array):
    offset = 0
    playlist = sp.playlist_tracks(playlist_id)
    for i in range(playlist["total"]):
        j = i%100
        print(i,j)
        if i == 99:
            offset += 1
            #rint("i: ",i)
            #rint("offset: ", offset+i)
            playlist = sp.playlist_tracks("2IuD0qZb14cji5y52crdsO", offset=i+offset,limit=100)
        add_track(playlist["items"][j]["track"]["id"],songs_array)
    return
            #len(playlist["tracks"]["items"])#.keys()#["items"]
#playlist["tracks"]["next"]#.keys()

playlist = sp.playlist_tracks("2IuD0qZb14cji5y52crdsO", limit=100)
print(playlist["items"][23]["track"]["id"])
#playlist2 = sp.playlist_tracks("2IuD0qZb14cji5y52crdsO", offset=101,limit=100)
#print("\n")
#print(playlist2["items"][23])
songs_array = add_songs_of_playlist(PLAYLIST_ID,songs)


3KAzDoRRG9BFI5DJ5JA6T1
0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 0
101 1
102 2
103 3
104 4
105 5
106 6
107 7
108 8
109 9
110 10
111 11
112 12
113 13
114 14
115 15
116 16
117 17
118 18
119 19
120 20
121 21
122 22
123 23
124 24
125 25
126 26
127 27
128 28
129 29
130 30
131 31
132 32
133 33
134 34
135 35
136 36
137 37
138 38
139 39
140 40
141 41
142 42
143 43
144 44
145 45
146 46
147 47
148 48
149 49
150 50
151 51
152 52
153 53
154 54
155 55
156 56
157 57
1

1175 75
1176 76
1177 77
1178 78
1179 79
1180 80
1181 81
1182 82
1183 83
1184 84
1185 85
1186 86
1187 87


In [107]:
songs

[{'album_id': '1MQO4j8QExVgmnplbIodEU',
  'album_name': 'Arca',
  'audio_features': {'acousticness': 0.486,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7j9DYPyCuvSAtPcevpAkzb',
   'danceability': 0.161,
   'duration_ms': 233079,
   'energy': 0.482,
   'id': '7j9DYPyCuvSAtPcevpAkzb',
   'instrumentalness': 0.409,
   'key': 1,
   'liveness': 0.371,
   'loudness': -11.21,
   'mode': 0,
   'speechiness': 0.0472,
   'tempo': 161.092,
   'time_signature': 5,
   'track_href': 'https://api.spotify.com/v1/tracks/7j9DYPyCuvSAtPcevpAkzb',
   'type': 'audio_features',
   'uri': 'spotify:track:7j9DYPyCuvSAtPcevpAkzb',
   'valence': 0.0926},
  'song_id': '7j9DYPyCuvSAtPcevpAkzb',
  'song_name': 'Desafío'},
 {'album_id': '1MQO4j8QExVgmnplbIodEU',
  'album_name': 'Arca',
  'audio_features': {'acousticness': 0.57,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1cwTMSQeMaA9fVKEF1iWeD',
   'danceability': 0.23,
   'duration_ms': 216905,
   'energy': 0.434,
   'id': '1cwTM

## Lyrics

Decisión de diseño
El género es una lista de géneros.

Hay canciones que no tienen un género asociado, en ese caso la api devuelve un []. Cómo tratar ese caso?

Yo propongo usar el género del artista. 

Qué pasa si shakira y metallica colaboran? qué género le ponés? Una decision posible: los géneros de los dos.

distribución de los géneros en el dataset.

### 2) Elegir tres modelos de clasificadores multiclase

Aquí escogeremos tres modelos diferentes y luego compararemos su rendimiento para esta tarea. El procedimiento será el siguiente:
- Inicializar los modelos
- Entrenarlos usando el split **train** de los datos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 3) Informe: Comparar el rendimiento de los modelos

Una vez entrenados los tres modelos, compararemos su rendimiento:
- Correr los modelos usando el split **test**
- Obtener el reporte de clasificación y la matriz de confusión para cada modelo
- Graficar llevando a 2 dimensiones nuestro split **test** pintando con colores diferentes según la etiqueta correspondiente.
- Graficar de manera similar los resultados obtenidos con cada clasificador y sobre esto la función de clasificación obtenida.
- Guardar los modelos usando **pickle**
- Discutir los resultados obtenidos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 4) Tareas adicionales:

Estas tareas servirán para extrapolar un poco el trabajo básico, y también sumarán puntos extra. Deben elegir una o más de las siguientes:
-  Análisis sobre el balance de clases del dataset, balanceo usando **subsampling** u **oversampling** y comparación de resultados vs el modelo básico
- Optimización de hiperparámetros y comparación de resultados vs el modelo básico
- Graficar importancia de features
- Graficar correlación de features

**Recomendación:**
- Hacer varias ahora puede ahorrarles tiempo en el futuro